[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1QPsU3xbgoY25jiMgoWamYhxKLT_iMHYA#scrollTo=XDspGgUS55Xo)

# Codice per montare GDrive su GColab

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/MyDrive/DataProjectAI

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1A0bC_-pQ-oWUI2c5-EqQVAZ6051G0tKg/DataProjectAI
